In [ ]:
import pandas as pd
import plotly.graph_objects as go
from janitor import clean_names
from sklearn.metrics import roc_auc_score

state_to_code = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
}

# Load data
df = pd.read_excel("story 1.xlsx").drop("Index", axis=1)

# Clean columnn
df = clean_names(df)

df["state_code"] = df["state"].map(state_to_code)

# Calculate ROC AUC
auc = roc_auc_score(df["political_affiliation"], df["fed_fund_%_of_state_revenue"])
print(f"AUC: {auc:.3f}")
contribution = abs(auc - 0.5) * 2  # Scale to 0-1
print(f"Feature contribution: {contribution:.3f}")

# Create color mapping for categories
category_colors = {"Blue": "Blues", "Red": "Reds"}

# Create separate traces for each category
fig = go.Figure()

for category in df["political_affiliation"].unique():
    category_data = df[df["political_affiliation"] == category]

    fig.add_trace(
        go.Choropleth(
            locations=category_data["state_code"],
            z=category_data[
                "fed_fund_%_of_state_revenue"
            ],  # Intensity by federal funding
            locationmode="USA-states",
            colorscale=category_colors.get(category, "Blues"),
            name=category,
            text=category_data["state"],
            hovertemplate="<b>%{text}</b><br>"
            + f"{category}<br>"
            + "Federal Funding: %{z:.1f}%<extra></extra>",
            showscale=True,
            colorbar=dict(
                title=f"{category}<br>Fed Funding %",
                x=1.05
                + (list(df["political_affiliation"].unique()).index(category) * 0.20),
            ),
        )
    )

fig.update_layout(
    title=dict(
        text="Federal Funding Impact on States' Polictical Affiliation",
        subtitle=dict(
            text="Federal Aid correlates but does not dictate party affiliation"
        ),
    ),
    geo=dict(scope="usa"),
    width=1200,  # Wider to accommodate multiple colorbars
    height=600,
)

fig.show()

Class 0 will be: Blue
Class 1 will be: Red
AUC: 0.675
Feature contribution: 0.350
